# Краткое описание проекта

<b>Постановка задачи: </b>На основании данных о поведении клиентов, которые уже перешли на имеющиеся тарифы, необходимо построить модель, которая выберет подходящий тариф. 
<br>
<b>Цель проекта:</b> Найти модель с наибольшим значением accuracy на валидационой выборке, проверить ее на тестовой выборке и проверить ее на адекватность <br>
<b>Задачи проекта: Найти модель с наибольшим значением accuracy, и проверить ее пригодность на тестовой выборке</b><br>
    <li>Открыть датасет и изучить его</li>
    <li>Разбить датасет на 3 части: тренировочные данные, валидационные данные, тестовые данные</li>
    <li>Проверить 3 модели, меняя значения гиперпараметров, и найти модель с наибольшим значением accuracy</li>
    <li>Проверить полученную модель на тестовой выборке</li>
<b>Общая информация о данных:</b><br>
    Каждый объект в наборе - информация о поведении одного пользователя в месяц. Известно, что:<br>
    - сalls — количество звонков,<br>
    - minutes — суммарная длительность звонков в минутах,<br>
    - messages — количество sms-сообщений,<br>
    - mb_used — израсходованный интернет-трафик в Мб,<br>
    - is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).<br>


## Изучение файла и общей информаци о нем

In [1]:
#импортируем все необходимые библиотеки
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import warnings
import random
#Установим общее значение random_state для всех функций (создадим константу для передачи в качестве параметра функции)
RANDOM_STATE = 12345

In [2]:
#откроем файл и посмотрим на него и общую информацию о нем
users_beahaviour = pd.read_csv('/datasets/users_behavior.csv')
print('Первые 15 строк датасета: ')
display(users_beahaviour.head(10))
print()
print('Общая информация о датасете:')
print(users_beahaviour.info())
print()
print('уникальные значения в столбце is_ultra::')
print(users_beahaviour['is_ultra'].unique())

Первые 15 строк датасета: 


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0



Общая информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None

уникальные значения в столбце is_ultra::
[0 1]


<b>Вывод:</b><br>
Согласно условию задачи,нам необходимо предложить пользователям один из двух тарифов: Smart или Ultra. В столбце is_ultra как раз содержится 2 уникальных значения: 0 - тариф Smart и 1 - тариф Ultra. Таким образом, можно говорить о том, что прогнозируемые данные - категориальные и нам предстроит задача классификации. В соответствии с этим в блок импортов выше впишем все необходимые библиотеки и классы:<br>
<li>DecisionTreeClassifier</li>
<li>RandomForestClassifier</li>
<li>LogisticRegression</li>
Таким образом, прогнозируемый параметр - is_ultra.<br>
Все остальные столбцы в датасете - данные, на основании которых осуществляется прогноз. Согласно условию задачи, данные прошли предобработку, поэтому их можно использовать для построения моделей<br>
Однако, возникает вопрос относительно необходимости столбца calls в анализе. Во-первых, само по себе количество вызовов не дает никакой картины по тому, сколько именно минут пользователь потратил в месяц (можно сделать 10 звонков длительностью по часу, а можно 30 звонков длительностью в одну минуту). А во-вторых, у нас действительно есть данные об общей продолительности звонков в течение месяца, что более информативно. Воплне возможно, наличие данного столбца может негативно повлиять на качество модели: как добавить продолжительности работы, так и привести к переобучению, например. В работе будем строить модели как со столбцом calls, так и без него, после чего сравним результат и веберем лучший.<br>
Согласно полученной информации, разобьем данные на выборки.<br>

## Разбиение данных на выборки

In [3]:
#выделим отдельно features и target:
users_behaviour_features = users_beahaviour.drop('is_ultra',axis = 1)
users_behaviour_target = users_beahaviour['is_ultra']
#разобьем данные на тренировочную и валидационную выборки (60% - обучающая, 20% - валидационная, 20%  - тестовая)
users_beahaviour_features_train_valid, users_beahaviour_features_test, users_beahaviour_target_train_valid, users_beahaviour_target_test = train_test_split(users_behaviour_features, users_behaviour_target, test_size = 0.2, random_state = RANDOM_STATE)
users_beahaviour_features_train, users_beahaviour_features_valid, users_beahaviour_target_train, users_beahaviour_target_valid = train_test_split(users_beahaviour_features_train_valid, users_beahaviour_target_train_valid, test_size = 0.25, random_state = RANDOM_STATE)
#Выведем размеры полученных выборок:
print('Размер тренировочной выборки: ',users_beahaviour_features_train.shape, ', доля от общей выборки: {:.1%}'.format(users_beahaviour_features_train.shape[0] / len(users_beahaviour)))
print('Размер  валидационной выборки: ',users_beahaviour_features_valid.shape, ', доля от общей выборки: {:.1%}'.format(users_beahaviour_features_valid.shape[0] / len(users_beahaviour)))
print('Размер тестовой выборки: ',users_beahaviour_features_test.shape, ', доля от общей выборки: {:.1%}'.format(users_beahaviour_features_test.shape[0] / len(users_beahaviour)))

Размер тренировочной выборки:  (1928, 4) , доля от общей выборки: 60.0%
Размер  валидационной выборки:  (643, 4) , доля от общей выборки: 20.0%
Размер тестовой выборки:  (643, 4) , доля от общей выборки: 20.0%


In [4]:
#создадим отдельно тренировочную, валидационную и тестовую выборки для данных без столбца calls
users_beahaviour_features_train_wo_calls = users_beahaviour_features_train.drop('calls',axis = 1)
users_beahaviour_features_valid_wo_calls = users_beahaviour_features_valid.drop('calls',axis = 1)
users_beahaviour_features_test_wo_calls = users_beahaviour_features_test.drop('calls',axis = 1)

<b>Вывод:</b><br>
в связи с тем, что, по условию задачи, тестовая выборка должна быть выбрана из исходного датасета, все данные разделили согласно следующему сценарию: 60% - на обучающую выборку, 20% - на валидационную, 20% - на тестовую (обычно размеры тестовой и валидационной выборки равны, это наиболее часто используемый сценарий).<br>
Таким образом, данные разделены на выборки, можно обучать и проверять модели.

## Исследуйте модели

<b>Для начала рассмотрим модель дерева решений</b>. Для построения оптимальной модели дерева решений (чтобы снизить как склонность к переобучению, так и склонность к недообучению), проведем ряд экспериментов с решающим деревом: подберем оптимальную глубину max_depth и посчитаем accuracy для каждой. <br>
Для начала посчитаем accuracy для модели без ограничений по глубине: в этом случае модель будет максимально медленной, а также максимально склонной к переобучению, однако мы будем иметь представление о максимально возможной глубине, что позволит ориентироваться при подборе проверяемых значений гиперпараметра max_depth
Если дерево высокое, у модели склонность к переобучению; низкое — к недообучению.

In [5]:
#функция создания модели DecisionTreeClassifier (учитывает как данные без max_depth, так и с ним. )
def make_best_decision_tree_classifier(features_train, target_train, features_valid, target_valid, depth): 
    if depth != None: #если указано значение depth, до которого стоит провести проверку, чтобы найти значение с максимальным accuracy
        best_model = None
        best_depth = 0
        best_accuracy = 0
        for i in range(1,depth): #создаем модели до depth
            model = DecisionTreeClassifier(random_state = RANDOM_STATE, max_depth = i)
            model.fit(features_train, target_train)
            accuracy = model.score(features_valid, target_valid)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = model
                best_depth = i
        print('Наибольшее значение accuracy =',accuracy,' при max_depth = ', best_depth)
        return best_model
    else: #если не указано значение depth
        model = DecisionTreeClassifier(random_state = RANDOM_STATE)
        model.fit(features_train, target_train)
        accuracy = model.score(features_valid, target_valid)
        print('Значение accuracy =',accuracy,' при max_depth = None')
        return model

max_depth = 15
#Проверим модели на полных данных:    
print('Результаты для модели на полных данных: ')
DecisionTreeClassifier_model_without_max_depth =  make_best_decision_tree_classifier(users_beahaviour_features_train, users_beahaviour_target_train, users_beahaviour_features_valid, users_beahaviour_target_valid, None)      
print('Глубина дерева решений, для которого параметр max_depth = None', DecisionTreeClassifier_model_without_max_depth.get_depth())
best_DecisionTreeClassifier_model_with_max_depth =  make_best_decision_tree_classifier(users_beahaviour_features_train, users_beahaviour_target_train, users_beahaviour_features_valid, users_beahaviour_target_valid, max_depth)  
#Проверим модели на данных без столбца calls
print()
print('Результаты для модели без столбца calls:')
DecisionTreeClassifier_model_wo_calls_without_max_depth =  make_best_decision_tree_classifier(users_beahaviour_features_train_wo_calls, users_beahaviour_target_train, users_beahaviour_features_valid_wo_calls, users_beahaviour_target_valid, None)      
print('Глубина дерева решений, для которого параметр max_depth = None', DecisionTreeClassifier_model_wo_calls_without_max_depth.get_depth())
best_DecisionTreeClassifier_model_wo_calls_with_max_depth =  make_best_decision_tree_classifier(users_beahaviour_features_train_wo_calls, users_beahaviour_target_train, users_beahaviour_features_valid_wo_calls, users_beahaviour_target_valid, max_depth)  



Результаты для модели на полных данных: 
Значение accuracy = 0.7122861586314152  при max_depth = None
Глубина дерева решений, для которого параметр max_depth = None 28
Наибольшее значение accuracy = 0.7573872472783826  при max_depth =  7

Результаты для модели без столбца calls:
Значение accuracy = 0.6982892690513219  при max_depth = None
Глубина дерева решений, для которого параметр max_depth = None 27
Наибольшее значение accuracy = 0.7371695178849145  при max_depth =  7


Можно видеть, что accuracy выше для модели на полных данных, причем max_depth при наибольшем значении accuracy одинаков для обеих моделей и равен 7. Соответственно, примем это значение за оптимальное 

<b>Проверим модель RandomForestClassifier</b>. Проведем эксперименты с количеством деревьев от 1 до 10. Для каждого из них проведем эксперимент по поиску оптимальной глубины max_depth. Это позволит построить модель с наибольшим значением accuracy

In [6]:
# Функция для создания моделей RandomForestClassifier и определения лучшей
def make_best_decision_forest_classifier(features_train, target_train, features_valid, target_valid, estimators, depth):
    best_model = None
    best_estimators = 0
    best_depth = 0
    best_accuracy = 0
    for i in range(1,estimators):
        if depth !=None:
            for j in range(1,depth):
                model = RandomForestClassifier(random_state = RANDOM_STATE, max_depth = j, n_estimators = i)
                model.fit(features_train, target_train)
                accuracy = model.score(features_valid, target_valid)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_model = model
                    best_depth = j
                    best_estimators = i
            print('Наибольшее значение accuracy =',accuracy,' при max_depth = ', best_depth, 'и n_estimators = ',best_estimators)
            return best_model
        else:
            model = RandomForestClassifier(random_state = RANDOM_STATE, max_depth = depth, n_estimators = i)
            model.fit(features_train, target_train)
            accuracy = model.score(features_valid, target_valid)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = model
                best_estimators = i
        print(print('Наибольшее значение accuracy =',accuracy,' при depth = ', depth, 'и n_estimators = ',best_estimators))
        return best_model

#зададим значения гиперпараметров:
estimators = 11
max_depth = 10
#Проверим модели на полных данных:    
print('Результаты для модели на полных данных: ')
ForestClassifier_model_without_max_depth =  make_best_decision_forest_classifier(users_beahaviour_features_train, users_beahaviour_target_train, users_beahaviour_features_valid, users_beahaviour_target_valid, estimators,None)      
best_ForestClassifier_model_with_max_depth =  make_best_decision_forest_classifier(users_beahaviour_features_train, users_beahaviour_target_train, users_beahaviour_features_valid, users_beahaviour_target_valid, estimators ,max_depth)  
print()
print('Результаты для модели на данных без calls: ')
ForestClassifier_model_wo_calls_without_max_depth =  make_best_decision_forest_classifier(users_beahaviour_features_train_wo_calls, users_beahaviour_target_train, users_beahaviour_features_valid_wo_calls, users_beahaviour_target_valid, estimators,None)      
best_ForestClassifier_model_wo_calls_with_max_depth =  make_best_decision_forest_classifier(users_beahaviour_features_train_wo_calls, users_beahaviour_target_train, users_beahaviour_features_valid_wo_calls, users_beahaviour_target_valid, estimators ,max_depth)  



Результаты для модели на полных данных: 
Наибольшее значение accuracy = 0.702954898911353  при depth =  None и n_estimators =  1
None
Наибольшее значение accuracy = 0.776049766718507  при max_depth =  7 и n_estimators =  1

Результаты для модели на данных без calls: 
Наибольшее значение accuracy = 0.7356143079315708  при depth =  None и n_estimators =  1
None
Наибольшее значение accuracy = 0.7713841368584758  при max_depth =  9 и n_estimators =  1


В данном случае также наибольшее значение accuracy достигается на модели для полных данных с n_estimators = 1 и max_depth = 7

<b>Проверим модель логистической регрессии</b>

In [7]:
#функция для создания логистической регрессии
def make_best_decision_logistic_regression(features_train, target_train, features_valid, target_valid):
    model=LogisticRegression(random_state = RANDOM_STATE)
    model.fit(features_train, target_train)
    accuracy = model.score(features_valid, target_valid)
    print('Accuracy модели = ', accuracy)
    return model

warnings.filterwarnings("ignore")

#Проверим модели на полных данных:    
print('Результаты для модели на полных данных: ')
LogisticRegression_model =  make_best_decision_logistic_regression(users_beahaviour_features_train, users_beahaviour_target_train, users_beahaviour_features_valid, users_beahaviour_target_valid)      
print()
print('Результаты для модели на данных без calls: ')
LogisticRegression_model_wo_calls =  make_best_decision_logistic_regression(users_beahaviour_features_train_wo_calls, users_beahaviour_target_train, users_beahaviour_features_valid_wo_calls, users_beahaviour_target_valid)      




Результаты для модели на полных данных: 
Accuracy модели =  0.6967340590979783

Результаты для модели на данных без calls: 
Accuracy модели =  0.7013996889580093


Результаты для модели логистической регрессии значительно ниже, чем для моделей "дерево решений" и "лес решений"

<b>Вывод:</b><br>
Сведем воедино все полученные значения accuracy:
<p align = left>
<table>
    <tr><td><b>Наименование и параметры модели</b></td><td><b>Accuracy для модели на полных данных</b></td><td><b>Accuracy для модели на данных без calls</b></td></tr>
    <tr><td>DecisionTreeClassifier max_depth = 7</td><td>0.7620528771384136</td><td>0.7573872472783826</td></tr>
    <tr><td>ForestClassifier n_estimators = 1 max_depth = 7 (9 для данных без calls)</td><td bgcolor = ffaa00>0.776049766718507</td><td bgcolor = #ffaa00>0.7713841368584758</td></tr>
    <tr><td>LogisticRegression</td><td>0.6967340590979783</td><td>0.7013996889580093</td></tr>
</table>    
</p>
Мы видим, что наибольшее значение accuracy достигается на модели ForestClassifier со значениями гиперпараметров max_depth = 7 и n_estimators = 1 (для модели с полными данными) и со значениями гиперпараметров max_depth = 9 и n_estimators = 1 (для модели с данными без calls)<br>
Ввиду того, что n_estimators = 1, скорость работы модели не будет замедлена слишком сильно, а полученные значения max_depth должны минимизировать риски переобучения. Accuracy для данных моделей максимально высокая. Поэтому для проверки на тестовой выборке используем именно эту модель<br>

## Проверьте модель на тестовой выборке

Проверим модель ForestClassifier со значениями гиперпараметров n_estimators = 1 max_depth = 7 на тестовой выборке для модели с полными данными

In [8]:
# best_ForestClassifier_model_with_max_depth - модель ForestClassifier со значениями гиперпараметров n_estimators = 1 max_depth = 7
users_beahaviour_test_predict = best_ForestClassifier_model_with_max_depth.predict(users_beahaviour_features_test)
accuracy_score = accuracy_score(users_beahaviour_target_test, users_beahaviour_test_predict)
print('Значение accuracy для тестовых данных (для модели на полных данных): ',accuracy_score)

Значение accuracy для тестовых данных (для модели на полных данных):  0.7869362363919129


Проверим модель ForestClassifier со значениями гиперпараметров n_estimators = 1 max_depth = 9 на тестовой выборке для модели с  данными без calls

In [9]:
users_beahaviour_test_predict_wo_calls = best_DecisionTreeClassifier_model_wo_calls_with_max_depth.predict(users_beahaviour_features_test_wo_calls)
accuracy_score = best_DecisionTreeClassifier_model_wo_calls_with_max_depth.score(users_beahaviour_features_test_wo_calls,users_beahaviour_target_test)
print('Значение accuracy для тестовых данных (для модели на данных без calls): ',accuracy_score)

Значение accuracy для тестовых данных (для модели на данных без calls):  0.7713841368584758


<b>Вывод</b><br>
Значение accuracy для тестовых данных осталость по-прежнему высоким, даже выше, чем для валидационных данных. Значение accuracy для модели на полных данных выше, чем для модели на данных без calls. По проведенным тестам модель на полных данных более точная. Осталось провести проверку модели на адекватность

<h2>Общий вывод:</h2>

Наиболее высокое значение accuracy покзала модель ForestClassifier на полных данных при max_depth = 7 и n_estimators = 1. Accuarcy данной модели на валидационной выбоке 77,6%, на тестовой - 78,69%.
Все значения accuracy для модели данных без calls заметно ниже, чем для полных данных, что говорит о том, что модель на полных данных более точна. 